# Série 1: Formalisation
## Les trois médecins

### Description
Alice, Bob et Charles sont trois patients suivis par les médecins Xavier,
Yolande et Zoé. Aujourd’hui, ces 6 personnes doivent être transférées de
l’Hôpital Guéritou vers l’Hôpital Soignetou grâce à une ambulance proposant
deux places. Les médecins ne doivent jamais se retrouver en infériorité
numérique pour prodiguer leur soins. On peut toutefois avoir des patients
dans un hôpital sans médecins. De plus l’ambulance ne fait pas de trajet à
vide.
Comment va-t-il falloir organiser les diférents trajets pour transporter
les 6 personnes d’un hôpital à l’autre, en respectant les contraintes ?

### Formalisation du problème recherche
Formalisez le problème en extrayant l’information suivante pour représenter
les différentes situations possibles. Profitez de cet exercice pour reviser les
notions du cours.

1. Donnez une représentation des états.

On ne considère pas que le fait que les médecins et les patients aient un nom soit important

On pourrait représenter un état du système par un tuple de la forme (x, y, z) avec x, le nombre de médecins dans l'Hôpital Guéritou,
y le nombre de patients dans l'Hôpital Guéritou, et z la position de l'ambulance... Pour connaître le nombre de patients et de médecins dans
l'Hôpital Soignetou, il suffit de faire respectivement le nombre total de médecins moins le nombre de médecins dans l'Hôpital Guéritou et le nombre total
de patients moins le nombre de patients dans l'Hôpital Soignetou.

L'ambulance est soit à l'Hôpital Guéritou, soit à l'Hôpital Soignetou. Pour modéliser le problème, on ne va donc pas considérer le temps de trajet car cela n'apporterait aucune valeur
ajoutée...

Dans notre cas, on a 3 médecins et 3 patients. L'espace des états possibles du système est donc donné par $S = (x, y, z)$ avec $x, y \in \{0, 1, 2, 3\}$ car il y a 3 médecins
et 3 patients, $x \geq y$ ou $x = 0$ car les médecins ne doivent jamais être en infériorité numérique pour prodiguer leurs soins et qu'on peut avoir des patients dans un
hôpital sans médecins, et $z \in \{0, 1\}$ car l'ambulance est soit à l'Hôpital Guéritou, soit dans l'Hôpital Soignetou. Comme notre référentiel est l'Hôpital Guéritou, l'ambulance est
soit là, soit pas là.

On peut définir l'espace d'états par: W = ({0-3}, {0-3}, {0-1})

2. Quels sont les opérateurs de transition possibles ?

Les opérateurs de transition possibles sont représentés par la fonction de transition:
$$\Gamma : S \rightarrow S$$
La fonction de transition est la contrainte de l'ambulance: l'ambulance a 2 places et ne fait jamais de trajets à vide.

On a donc soit un patient, soit un médecin, soit un patient et un médecin, soit deux médecins, soit deux patients dans l'ambulance. Ce sont nos opérateurs de transition possibles.

3. Définissez les conditions pour lesquelles les opérateurs sont applicables.

Les opérateurs sont applicables si on peut passer d'un état appartenant à l'espace d'états du système à un autre espace appartenant à l'espace d'états du système.
Donc les opérateurs sont applicables si:
$ f (x, y, z) = (x', y', z')$ avec $x', y' \in \{0, 1, 2, 3\}$, si ($x' \geq y'$ ou $x'=0$).

On a également que
- si z' = 1, alors $x'+y' - (x+y) \in \{1, 2\}$ et $x' - x, y'-y \in \{1, 2\}$
- si z'= 0, alors $x+y - (x'+y') \in \{1, 2\}$ et $x - x', y - y' \in \{1, 2\}$

Cette condition oblige à avoir minimum une personne dans l'ambulance et maximum deux personnes dans l'ambulance, et à ne pas avoir des téléportations de personnes. 

(La deuxième partie a été ajoutée après avoir fait la question 4. car je me suis retrouvé devant le cas de téléportation de personnes: (3, 2, 1) $\rightarrow$ (0, 3, 0). Dans ce cas, 
la contrainte de l'ambulance est respectée,
ainsi que la contrainte $x \geq y$ ou $x=0$)

4. En vous basant sur le _TP0_, implementez un algorithme de recherche
pour résoudre le problème en utilisant un arbre de recherche correspondant à la description que vous avez choisi.

In [1]:
import numpy as np

def list_possibilities(node: list[int]) -> list[list[int]]:
    k = 0
    if node[2] == 0:
        k = 1
    result = []
    for i in np.arange(4):
        for j in np.arange(4):
            if ispossible(node, [i, j, k]):
                result.append([i, j, k])
    return result
                

# Node: doctors, cobayes, cars

def ispossible(node: list[int], next: list[int]) -> bool:
    if node[2] != next[2]:
        var = 0
        if (next[0] >= next[1] or next[0] == 0):
            if node[2] == 0:
                var = next[0] + next[1] - (node[0] + node[1])
                if (next[0] - node[0]) > 2 or (next[1] - node[1]) > 2: return False
            else:
                var = node[0] + node[1] - (next[0] + next[1])
                if (node[0] - next[0]) > 2 or (node[1] - next[1]) > 2: return False
        if (var == 1 or var == 2):
            return True
    return False

def stop_condition(node: list[int]) -> bool:
    if (node[0] != 0 or node[1] != 0):
        return False
    return True

def find_parent(node: list[int], list_parents: list[list[list[int]]]) -> list[int]:
    for i in range(len(list_parents)):
        if node == list_parents[i][0]:
            return list_parents[i][1]
    return []

def BFS(start: list[int]):
    stack = []
    node = []
    parents = []
    path = []
    stack.append(start)
    visited = []
    while len(stack) != 0:
        node = stack.pop(0)
        visited.append(node)
        if stop_condition(node):
            break
        tmp = list_possibilities(node)
        for i in tmp:
            if i not in visited:
                stack.append(i)
                parents.append([i, node])
    if stop_condition(node):
        path.append(node)
        parent = find_parent(node, parents)
        while parent != []:
            path.append(parent)
            parent = find_parent(parent, parents)
        return path[::-1]
    else:
        print("No solution")

print(BFS([3, 3, 1]))

[[3, 3, 1], [2, 2, 0], [3, 2, 1], [2, 1, 0], [2, 2, 1], [0, 2, 0], [0, 3, 1], [0, 1, 0], [0, 2, 1], [0, 0, 0]]


### Complexité

1. Calculez la taille de l’espace de recherche. Pour taille de l’espace de
recherche on considère tout l’espace de recherche, y compris les états
où les patients ne sont plus accompagnés.

La taille de l'espace de recherche est donné par:

Taille de S: 
$$S = 4 \times 4 \times 2 - 1 - 1= 30$$

En effet, comme on considère tout l'espace de recherche, y compris les états où les patients ne sont plus accompagnés, on considère qu'il peut y avoir moins de médecins que de patients car le patient n'a
plus besoin d'être accompagné et suivi par un médecin. Auquel cas, on a simplement le choix entre 0, 1, 2, ou 3 médecins, le choix entre 0, 1, 2, ou 3 patients et le choix de la position de l'ambulance qui est soit 0, soit 1. Cependant, on soustrait le cas où on est dans l'état (3, 3, 0) et l'état (0, 0, 1), car cela impliquerait que l'ambulance fasse un trajet à vide, ce qui n'est pas possible...

2. Calculez le nombre d’états où chaque patient est accompagné.

Le nombre d'états où chaque patient est accompagné est égal à:

Nombre d'états de S moins la possibilité qu'il y ait 0 médecins dans l'hôpital Guéritou alors que l'ambulance est là ($\sum_{i=1}^{4} 1 = 3$), moins la possibilité qu'il y ait des patients pas
accompagnés alors que l'ambulance n'est pas là ($\sum_{i=1}^{4} 1 = 3$), moins la possibilité qu'il y ait moins de médecins que de patients ($\sum_{i = 1}^{3}(\sum_{j = i + 1}^{4} 1)$. Cette formule
donne la probabilité qu'il y ait 1 ou 2 docteurs pour un nombre de patients compris entre nombre de docteurs + 1 et nombre maximal de patients...). Ce qui donne:

Nombre d'états où chaque patient est accompagné:

$$AC = 30 - 2\times\sum_{i=1}^{4} 1 - \sum_{i = 1}^{3}(\sum_{j = i + 1}^{4} 1)
= 30 - 6 - \sum_{i = 0}^{2}(\sum_{j = i + 1}^{3}1)
= 24 - 2\times3
= 18$$

In [3]:
def size_search_space(nodes: list[list[int]], current: list[int]) -> list[list[int]]:
    nodes.append(current)
    possibilities = list_possibilities(current)
    for i in possibilities:
        if i not in nodes:
            size_search_space(nodes, i)
    return nodes

def patients():
    tuples = size_search_space([], [3, 3, 1])
    n = len(tuples)
    for i in range(1, 4):
        if [0, i, 0] in tuples:
            n -= 1
    for i in range(3):
        if [3, i, 1] in tuples:
                n -= 1
    return n

print(patients())
        

18


3. Calculez le nombre d’états accessibles depuis l’état initial.

Le nombre d'états accessibles depuis l'état initial est donné par le nombre d'états total moins le nombre d'états où les patients ne sont pas accompagnés plus le nombre d'états où on a 0 médecins, des patients et l'ambulance à un hôpital ($\sum_{i = 1}^{4} 1$) fois deux car cela peut s'appliquer aux deux hôpitaux. Cela nous donne:

$$accessible = S - (S - AC) + (\sum_{i = 1}^{4} 1)\times2
= 30 - (30 - 18) + 3\times2
= 24$$

In [ ]:
print(len(size_search_space([], [3, 3, 1])))

## Les tours de Hanoi

### Description

Les tours de Hanoi est un casse-tête dont le but est de transférer une tour
d’un poteau à l’autre. Le jeu se compose de n disques de tailles croissante
enilés sur 3 poteaux. Au début du jeu, les n disques forment une tour
croissante sur le poteau de gauche. Le but est de transférer cette tour sur
le poteau de droite en utilisant aussi le poteau du milieu. On ne peut pas
empiler un disque sur un autre plus petit.

### Questions
1. Formalisez le problème:
   - formalisation d'un état
   - formalisation de l'état initial et de l'état final
   - formalisation des transitions

- On pourrait représenter un état par une liste de liste $[[x_1, \dots, x_l], [y_1, \dots, y_m], [z_1, \dots, z_p]]$ avec $x_1, \dots, x_l , y_1, \dots, y_m$ et $z_1, \dots, z_p$ la taille des disques présents sur les tours $x, y,$ et $z$. On a que le nombre de disques total est égal à $l + m + p$. Donc pour $n$ disques, on a:
$n = l + m + p$

    [] représente l'état où la tour ne possède aucun disque...
    
    La fin de chaque liste (donc ici $x_l, y_m, z_p$) représente le disque du sommet de la tour $x, y$ et $z$.

    On est contrait à avoir une représentation comme cela de nos états, car l'information de quel disque se trouve sur quel disque est importante pour savoir quel disque va se trouver en sommet de tour si
    on enlève un disque...

- L'état initial est donné par $[[], [], [z_1, \dots, z_n]]$ avec n le nombre de disques total trié dans l'ordre du plus grand disque au plus petit disque (fin de liste = disque du sommet de la pile)

    L'état final est donné par $[[x_1, \dots, x_n], [], []]$

- On peut définir la fonction de transition comme:
$f: S \rightarrow S$
tel que $f(state) = newstate$ avec $state$ et $newstate$ une liste de liste représentant un état du système, comme définie précédement.

    On aura comme contrainte que si le disque de la tour $i$ a été déplacé sur la tour $j$, alors:
    - $last.element(state[i]) = last.element(newstate[j])$
    - $last.element(state[i]) \lt last.element(state[j])$ OR $state[j]$ is empty

2. Combien y a-t-il d'états possibles pour $n = 3$ ?

On a:
- $1\times3$ correspondant au nombre d'états où les 3 disques sont empilés sur une tour
- $3\times2\times1$ correspondant au nombre d'états où chaque disque est sur une tour différente
- $(3\times2\times1)\times3$ correspondant au nombre d'états où une tour contient 2 disques sachant qu'il y a à chaque fois 2 agencements possibles de disques pour mettre 2 disques sur une tour...

Ce qui donne un nombre d'états possible de:
$$1\times3 + 3\times2\times1 + (3\times2\times1)\times3 = 27\ états\ possibles$$

3. En vous basant sur le TP0, implémentez la solution pour $n=3$

In [2]:
import copy as cp

def is_possible(state: list[list[int]], move: list[int]) -> bool:
    if len(state[move.index(1)]) == 0: return False
    if len(state[move.index(2)]) == 0: return True
    if state[move.index(1)][0] < state[move.index(2)][0]: return True
    return False

def execute_possibilitie(state: list[list[int]], move: list[int]) -> list[list[int]]:
    disk = state[move.index(1)].pop()
    state[move.index(2)].append(disk)
    return state

def list_possibilities(state: list[list[int]]) -> list[list[int]]:
    possibilities = []
    for i in range(3):
        for j in range(3):
            for k in range(3):
                if i != j and j != k and i != k:
                    if (is_possible(state, [i, j, k])):
                        newstate = cp.deepcopy(state)
                        possibilities.append(execute_possibilitie(newstate, [i, j, k]))
    return possibilities



def stop_condition(state: list[list[int]]) -> bool:
    if state[0] == [2, 1, 0] and len(state[1]) + len(state[2]) == 0: return True
    return False


def find_parent(node: list[int], list_parents: list[list[list[int]]]) -> list[int]:
    for i in range(len(list_parents)):
        if list_parents[i][0][0] == node[0] and list_parents[i][0][1] == node[1] and list_parents[i][0][2] == node [2]:
            return list_parents[i][1]
    return []

def is_in_list(state: list[list[int]], visited: list[list[list[int]]]) -> bool:
    for i in range(len(visited)):
        if visited[i][0] == state[0] and visited[i][1] == state[1] and visited[i][2] == state[2]: return True
    return False



def BFS_2(state: list[list[int]]) -> list[list[int]]:
    stack = []
    node = []
    parents = []
    visited = []
    path = []
    stack.append(state)
    while len(stack) != 0:
        node = stack.pop(0)
        visited.append(node)
        if stop_condition(node):
            break
        tmp = list_possibilities(node)
        for i in tmp:
            if not is_in_list(i, visited):
                stack.append(i)
                parents.append([i, node])
    if stop_condition(node):
        path.append(node)
        parent = find_parent(node, parents)
        while parent != []:
            path.append(parent)
            parent = find_parent(parent, parents)
        return path[::-1]
    else:
        print("No result")
        return []

print(BFS_2([[], [], [2, 1, 0]]))
                    

[[[], [], [2, 1, 0]], [[0], [], [2, 1]], [[0], [1], [2]], [[], [1, 0], [2]], [[2], [1, 0], []], [[2], [1], [0]], [[2, 1], [], [0]], [[2, 1, 0], [], []]]
